In [1]:
# Dependencies, load file and look at what we working with
import pandas as pd
purchase_data = pd.read_csv("Resources/purchase_data.csv")
#Optional: give the displayed data cleaner formatting - that way I only need to change it for %s couple of times.
pd.options.display.float_format = '${:,.2f}'.format
purchase_data.head(1)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",$3.53


## Player Count

In [2]:
#Display the total number of players
group_byname = purchase_data.groupby('SN')
number_ofPurchases = group_byname['SN'].count()
print('''Total number of players that bougt something:''', len(number_ofPurchases))

Total number of players that bougt something: 576


## Purchasing Analysis (Total)

In [3]:
group_byname = purchase_data.groupby('Item Name')
number_ofItems = group_byname['Item Name'].count()
print('''Number of Unique Items: ''', len(number_ofItems))
print('''Average Price: $''',round(purchase_data['Price'].mean(),2))
print('''Number of Purchases: ''', len(purchase_data['Purchase ID']))
print('''Total Revenue: $''', round(purchase_data['Price'].sum(),2))

Number of Unique Items:  179
Average Price: $ 3.05
Number of Purchases:  780
Total Revenue: $ 2379.77


## Gender Demographics

In [4]:
group_nameNgender = purchase_data.groupby(['SN','Gender']).count()
byGender = group_nameNgender.groupby('Gender').count()
del byGender['Item ID']
del byGender['Item Name']
del byGender['Price']
byGender.columns = ['Total Count','Percentage of Players']
byGender.iloc[0,1] = byGender.iloc[0,0] / len(number_ofPurchases) * 100
byGender.iloc[1,1] = byGender.iloc[1,0] / len(number_ofPurchases) * 100
byGender.iloc[2,1] = byGender.iloc[2,0] / len(number_ofPurchases) * 100
byGender['Percentage of Players'] = byGender['Percentage of Players'].apply('{:,.2f}%'.format)
byGender

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

In [5]:
tmp1 = purchase_data.groupby('SN').agg({'Purchase ID':'count','Gender':'max','Price':'sum'})
by_gender = tmp1.groupby('Gender').agg({'Purchase ID':'sum','Price':['mean','sum']})
by_gender['Average Purchase Price'] = 0
by_gender.columns = ['Purcase Count','Avg per person','Total purchases','Avg per purchase']
by_gender.iloc[0,3] = by_gender.iloc[0,2] / by_gender.iloc[0,0]
by_gender.iloc[1,3] = by_gender.iloc[1,2] / by_gender.iloc[1,0]
by_gender.iloc[2,3] = by_gender.iloc[2,2] / by_gender.iloc[2,0]
by_gender

,Purcase Count,Avg per person,Total purchases,Avg per purchase
Gender,,,,
Female,113,$4.47,$361.94,$3.20
Male,652,$4.07,"$1,967.64",$3.02
Other / Non-Disclosed,15,$4.56,$50.19,$3.35


## Age Demographics

In [6]:
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
group_names = [" <10", "10-14", "15-19", "20-24", "25-29", "30-34","35-39","40+"]
purchase_data["Age Group"] = pd.cut(purchase_data["Age"], bins, labels=group_names, include_lowest=True)
tmp1 = purchase_data.groupby('SN').agg({'Purchase ID':'count','Age Group':'max'})
group_byAge = tmp1.groupby('Age Group').count()
group_byAge.columns=['Total Count']
group_byAge['Percentage of Players'] = 0
for i in range(8):
    group_byAge.iloc[i,1] = group_byAge.iloc[i,0] / len(number_ofPurchases) * 100
group_byAge['Percentage of Players'] = group_byAge['Percentage of Players'].apply('{:,.2f}%'.format)
group_byAge

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [7]:
tmp1 = purchase_data.groupby('Age Group').agg({'Purchase ID':'count','Price':['mean','sum'],'SN':'count'})
tmp1.columns=['Purchase Count','Average Purchase Price','Total Purchase Value','Avg purchase per person']
for i in range(8):
    tmp1.iloc[i,3] = tmp1.iloc[i,2] / group_byAge.iloc[i,0]
tmp1

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg purchase per person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

In [8]:
tmp1 = purchase_data.groupby('SN').agg({'Purchase ID':'count','Price':['mean','sum']})
tmp1.columns=['Purchase Count','Average Purchase Price','Total Purchase Value']
tmp2=tmp1.sort_values('Total Purchase Value',ascending=False,)
tmp2.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [9]:
tmp1 = purchase_data.groupby('Item ID').agg({'Item Name':'max','Purchase ID':'count','Price':['max','sum']})
tmp1.columns=['Item Name','Purchase Count','Item Price','Total Purchase Value']
tmp2=tmp1.sort_values('Purchase Count',ascending=False,)
tmp2.head()

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.33,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

In [10]:
tmp1=tmp2.sort_values('Total Purchase Value',ascending=False,)
tmp1.head()

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
